In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
data_mgt = fetch_ucirepo(id=159) 

# data (as pandas dataframes) 
X_mgt = data_mgt.data.features
y_mgt = data_mgt.data.targets

In [ ]:
print(y_mgt)

In [ ]:
# ==========================================
# Divisão Treino/Teste e Normalização
# ==========================================

from sklearn.preprocessing import StandardScaler, LabelBinarizer

lb = LabelBinarizer()
y_bin = lb.fit_transform(y_mgt).ravel()

# 1. Divisão Hold-out (70% Treino, 30% Teste)
# 'stratify=y' garante que mantemos a proporção de doentes/saudáveis em ambos os conjuntos
X_train, X_test, y_train, y_test = train_test_split(X_mgt, y_bin, test_size=0.3, random_state=42, stratify=y_bin)

# 2. Normalização (StandardScaler)
# IMPORTANTE: O KNN e a Regressão Logística são sensíveis à escala dos dados.
# O Naive Bayes (Gaussiano) lida bem sem, mas mal não faz neste contexto comparativo.
scaler = StandardScaler()

# Ajustamos (fit) apenas ao TREINO para evitar vazamento de dados (data leakage)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dados divididos e normalizados.")
print(f"Treino: {X_train_scaled.shape[0]} amostras")
print(f"Teste: {X_test_scaled.shape[0]} amostras")

In [ ]:
print(y_bin)
print(X_test_scaled)

In [ ]:
# --- RAMO 1: Random Forest (Bagging) ---
# Limitada (max_depth=2). Tenta aproximar círculo com retângulos.
# Resultado: Fronteira quadrada/diamante.
model_paralelo = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_paralelo.fit(X_train_scaled, y_train)
acc_paralelo = model_paralelo.score(X_test_scaled, y_test)

# Cálculo da AUC
y_prob = model_paralelo.predict_proba(X_test_scaled)

# Se for um problema binário (2 classes) então selecionar apenas a classe positiva
if y_prob.shape[1] == 2:
    y_prob = y_prob[:,1]

auc_paralelo = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')

In [ ]:
print(y_prob)

In [ ]:
# --- RAMO 2: Gradient Boosting (Boosting) ---
# Sequencial. Corrige os erros de forma (viés).
# Resultado: Círculo quase perfeito.
model_sequencial = GradientBoostingClassifier(n_estimators=100, max_depth=10, random_state=42)
model_sequencial.fit(X_train_scaled, y_train)
acc_sequencial = model_sequencial.score(X_test_scaled, y_test)

# Cálculo da AUC
y_prob = model_sequencial.predict_proba(X_test_scaled)

# Se for um problema binário (2 classes) então selecionar apenas a classe positiva
if y_prob.shape[1] == 2:
    y_prob = y_prob[:,1]

auc_sequencial = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')

In [ ]:
# --- RAMO 3: SVM Linear ---
modelo_svm_linear = SVC(kernel='linear', probability=True, random_state=42) # Outro linear!
modelo_svm_linear.fit(X_train_scaled, y_train)
acc_svm_linear = modelo_svm_linear.score(X_test_scaled, y_test)

# Para SVM, usamos decision_function para obter o "score" (distância da margem)
y_score = modelo_svm_linear.decision_function(X_test_scaled)

# Cálculo da ROC e AUC
fpr, tpr, _ = roc_curve(y_test, y_score)
auc_svm_linear = auc(fpr, tpr)

In [ ]:
# --- RAMO 4: SVM RBF ---
modelo_svm_rbf = SVC(kernel='rbf', probability=True, random_state=42) # Outro linear!
modelo_svm_rbf.fit(X_train_scaled, y_train)
acc_svm_rbf = modelo_svm_rbf.score(X_test_scaled, y_test)

# Para SVM, usamos decision_function para obter o "score" (distância da margem)
y_score = modelo_svm_linear.decision_function(X_test_scaled)

# Cálculo da ROC e AUC
fpr, tpr, _ = roc_curve(y_test, y_score)
auc_svm_rbf = auc(fpr, tpr)

In [ ]:
# 4. Visualização
print(f"Random Forest:     Accuracy = {acc_paralelo:.2f}  /  AUC = {auc_paralelo:.2f}")
print(f"Gradient Boosting: Accuracy = {acc_sequencial:.2f}  /  AUC = {auc_sequencial:.2f}")
print(f"SVM Linear:        Accuracy = {acc_svm_linear:.2f}  /  AUC = {auc_svm_linear:.2f}")
print(f"SVM RBF:           Accuracy = {acc_svm_rbf:.2f}  /  AUC = {auc_svm_rbf:.2f}")
